In [35]:
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn import linear_model,metrics,svm
from sklearn.ensemble import RandomForestRegressor
from geopy.distance import vincenty,great_circle
import xgboost as xgb
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
import requests
import time
import holidays

In [14]:
#import training and testing dataframe
train = pd.read_csv('../charlie/Desktop/NYC_Taxi/train.csv')
test = pd.read_csv('../charlie/Desktop/NYC_Taxi/test.csv')

#check the numbers of dataframe
if len(train) == 1458644:
    print 'numbers of training data :correct!'
else:
    print "numbers of training data :incorrect!"

if len(test) == 625134:
    print 'numbers of testing data :correct!'
else:
    print "numbers of testing data :incorrect!"
    
# for convinience,make them together to do futher process
df = pd.concat([train,test])
print len(df)
df.head()

numbers of training data :correct!
numbers of testing data :correct!


In [ ]:
# desccriptive statistics
print len(train.columns.values)
print len(test.columns.values)
print len(df.columns.values)

In [25]:
#time
df['p_date'] = pd.to_datetime(df['pickup_datetime'])
df['d_date'] = pd.to_datetime(df['dropoff_datetime'])
df['weekday'] = df.p_date.dt.weekday
df.loc[df.weekday >= 5, 'is_weekend'] = 1
df.loc[df.weekday < 5, 'is_weekend'] = 0
df['p_hour'] = df.p_date.dt.hour
df.loc[(df.p_hour >=0 ) & (df.p_hour <6),'time_period'] = 1
df.loc[(df.p_hour >=6 ) & (df.p_hour <12),'time_period'] = 2
df.loc[(df.p_hour >=12 ) & (df.p_hour <18),'time_period'] = 3
df.loc[(df.p_hour >=18 ) & (df.p_hour <24),'time_period'] = 4
df['month'] = df['p_date'].dt.month
df['weekofyear'] = df['p_date'].dt.weekofyear

#distance
def D(x,y,z,w):
    return vincenty((x, y),(z, w)).miles
df['distance'] = map(D,df.pickup_latitude,df.pickup_longitude,df.dropoff_latitude,df.dropoff_longitude)

#trip_duration
df['trip_duration2'] = np.log(df.trip_duration.values)
df.head(5)

,dropoff_datetime,dropoff_latitude,dropoff_longitude,id,passenger_count,pickup_datetime,pickup_latitude,pickup_longitude,store_and_fwd_flag,trip_duration,vendor_id,p_date,d_date,weekday,is_weekend,p_hour,time_period,month,weekofyear,distance
0,2016-03-14 17:32:30,40.765602,-73.964630,id2875421,1,2016-03-14 17:24:55,40.767937,-73.982155,N,455.0,2,2016-03-14 17:24:55,2016-03-14 17:32:30,0,0.0,17,3.0,3,11,0.933406
1,2016-06-12 00:54:38,40.731152,-73.999481,id2377394,1,2016-06-12 00:43:35,40.738564,-73.980415,N,663.0,1,2016-06-12 00:43:35,2016-06-12 00:54:38,6,1.0,0,1.0,6,23,1.123849
2,2016-01-19 12:10:48,40.710087,-74.005333,id3858529,1,2016-01-19 11:35:24,40.763939,-73.979027,N,2124.0,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,0.0,11,2.0,1,3,3.964154
3,2016-04-06 19:39:40,40.706718,-74.012268,id3504673,1,2016-04-06 19:32:31,40.719971,-74.010040,N,429.0,2,2016-04-06 19:32:31,2016-04-06 19:39:40,2,0.0,19,4.0,4,14,0.921886
4,2016-03-26 13:38:10,40.782520,-73.972923,id2181028,1,2016-03-26 13:30:55,40.793209,-73.973053,N,435.0,2,2016-03-26 13:30:55,2016-03-26 13:38:10,5,1.0,13,3.0,3,12,0.737591
5,2016-01-30 22:09:03,40.749184,-73.992081,id0801584,6,2016-01-30 22:01:40,40.742195,-73.982857,N,443.0,2,2016-01-30 22:01:40,2016-01-30 22:09:03,5,1.0,22,4.0,1,4,0.683275
6,2016-06-17 22:40:40,40.765896,-73.957405,id1813257,4,2016-06-17 22:34:59,40.757839,-73.969017,N,341.0,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,0.0,22,4.0,6,24,0.824764
7,2016-05-21 08:20:49,40.760559,-73.922470,id1324603,1,2016-05-21 07:54:58,40.797779,-73.969276,N,1551.0,2,2016-05-21 07:54:58,2016-05-21 08:20:49,5,1.0,7,2.0,5,20,3.553009
8,2016-05-27 23:16:38,40.732815,-73.985786,id1301050,1,2016-05-27 23:12:23,40.738400,-73.999481,N,255.0,1,2016-05-27 23:12:23,2016-05-27 23:16:38,4,0.0,23,4.0,5,21,0.815587
9,2016-03-10 22:05:26,40.789989,-73.973000,id0012891,1,2016-03-10 21:45:01,40.744339,-73.981049,N,1225.0,2,2016-03-10 21:45:01,2016-03-10 22:05:26,3,0.0,21,4.0,3,10,3.178194


In [30]:
# use Open Street Road Map(OSRM)
#import data
fr_train1 = pd.read_csv('../charlie/Desktop/NYC_Taxi/fastest_routes_train_part_1.csv')
fr_train2 = pd.read_csv('../charlie/Desktop/NYC_Taxi/fastest_routes_train_part_2.csv')
fr_test = pd.read_csv('../charlie/Desktop/NYC_Taxi/fastest_routes_test.csv')
fr_train = pd.concat([fr_train1,fr_train2])
print len(fr_train)
fr = pd.concat([fr_train,fr_test])
print len(fr)
df = df.merge(fr, how='left', on='id')
df['speed'] = df['total_distance']/df['total_travel_time']
print len(df)

1458643
2083777
2083778


In [39]:
# coordinate:dimention reduction
coords = np.vstack((df[['pickup_latitude', 'pickup_longitude']].values,
                    df[['dropoff_latitude', 'dropoff_longitude']].values))
pca = PCA().fit(coords)
df['pickup_pca0'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:,0]
df['pickup_pca1'] = pca.transform(df[['pickup_latitude', 'pickup_longitude']])[:,1]
df['dropoff_pca0'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:,0]
df['dropoff_pca1'] = pca.transform(df[['dropoff_latitude', 'dropoff_longitude']])[:,1]
#clustering
#sample_ind = np.random.permutation(len(coords))[:500000]
kmeans = MiniBatchKMeans(n_clusters=100, batch_size=10000).fit(coords)
df['pickup_group'] = kmeans.predict(df[['pickup_latitude', 'pickup_longitude']])
df['dropoff_group'] = kmeans.predict(df[['dropoff_latitude', 'dropoff_longitude']])

In [40]:
#split data into training and testing data


,dropoff_datetime,dropoff_latitude,dropoff_longitude,id,passenger_count,pickup_datetime,pickup_latitude,pickup_longitude,store_and_fwd_flag,trip_duration,...,step_maneuvers,step_direction,step_location_list,speed,pickup_pca0,pickup_pca1,dropoff_pca0,dropoff_pca1,pickup_group,dropoff_group
0,2016-03-14 17:32:30,40.765602,-73.964630,id2875421,1,2016-03-14 17:24:55,40.767937,-73.982155,N,455.0,...,depart|rotary|turn|new name|arrive,left|straight|right|straight|arrive,"-73.982316,40.767869|-73.981997,40.767688|-73....",12.183748,0.007691,0.017053,-0.009666,0.013695,33,19
1,2016-06-12 00:54:38,40.731152,-73.999481,id2377394,1,2016-06-12 00:43:35,40.738564,-73.980415,N,663.0,...,depart|turn|turn|end of road|continue|arrive,none|right|left|right|left|arrive,"-73.980429,40.73857|-73.985444,40.731658|-73.9...",7.569880,0.007677,-0.012371,0.027145,-0.018652,7,36
2,2016-01-19 12:10:48,40.710087,-74.005333,id3858529,1,2016-01-19 11:35:24,40.763939,-73.979027,N,2124.0,...,depart|turn|turn|turn|new name|turn|on ramp|me...,right|left|right|left|straight|right|straight|...,"-73.978874,40.764148|-73.977685,40.763646|-73....",14.409588,0.004803,0.012879,0.034222,-0.039337,0,18
3,2016-04-06 19:39:40,40.706718,-74.012268,id3504673,1,2016-04-06 19:32:31,40.719971,-74.010040,N,429.0,...,depart|turn|end of road|arrive,left|left|right|arrive,"-74.010145,40.719982|-74.011527,40.714294|-74....",7.546226,0.038342,-0.029194,0.041343,-0.042293,78,38
4,2016-03-26 13:38:10,40.782520,-73.972923,id2181028,1,2016-03-26 13:30:55,40.793209,-73.973053,N,435.0,...,depart|turn|turn|turn|arrive,right|left|right|left|arrive,"-73.972998,40.793187|-73.976607,40.788361|-73....",11.526767,-0.002877,0.041749,-0.002380,0.031071,75,69
